<a href="https://colab.research.google.com/github/RaquelFonsec/machine-learning/blob/main/machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# 🟢 1. Tentar detectar automaticamente o separador correto
with open("/content/Desempenho dos estudantes em Portugal - matemática.csv", "r", encoding="utf-8") as f:
    primeira_linha = f.readline()
    print("Primeira linha do CSV:", primeira_linha)

# 🟢 2. Tentar carregar com diferentes separadores
sep_possiveis = [";", ",", "\t"]  # Testar ponto e vírgula, vírgula e tabulação
for sep in sep_possiveis:
    df = pd.read_csv("/content/Desempenho dos estudantes em Portugal - matemática.csv", sep=sep, engine="python")

    # Se o dataframe tiver mais de uma coluna,
    if df.shape[1] > 1:
        print(f"✅ Separador correto identificado: '{sep}'")
        break
else:
    print("❌ Nenhum separador funcionou corretamente. Verifique o arquivo!")

# 🟢 3. Exibir as colunas corrigidas
print("✅ Colunas do DataFrame corrigidas:", df.columns)
print(df.head())  # Exibir as primeiras linhas para verificar

# 🟢 4. Verificar se a coluna 'nota_final' está presente
if "nota_final" not in df.columns:
    print("⚠️ A coluna 'nota_final' ainda não foi encontrada. Verifique se o nome está correto!")
    print("Colunas disponíveis:", df.columns)




Primeira linha do CSV: escola,sexo,idade,tipo_endereco,tamanho_familia,status_pais,educ_mae,educ_pai,trab_mae,trab_pai,motivo_escolha,responsavel,tempo_viagem,tempo_estudo,num_reprovacoes,apoio_escola,apoio_familia,aulas_particulares,atividades_extracurriculares,frequentou_creche,deseja_superior,acesso_internet,relacionamento_romantico,qualidade_rel_familiar,tempo_livre,saidas,consumo_alcool_dia,consumo_alcool_fimsemana,estado_saude,num_faltas,nota_periodo1,nota_periodo2,nota_final,#N/D

✅ Separador correto identificado: ','
✅ Colunas do DataFrame corrigidas: Index(['escola', 'sexo', 'idade', 'tipo_endereco', 'tamanho_familia',
       'status_pais', 'educ_mae', 'educ_pai', 'trab_mae', 'trab_pai',
       'motivo_escolha', 'responsavel', 'tempo_viagem', 'tempo_estudo',
       'num_reprovacoes', 'apoio_escola', 'apoio_familia',
       'aulas_particulares', 'atividades_extracurriculares',
       'frequentou_creche', 'deseja_superior', 'acesso_internet',
       'relacionamento_romantico', '

In [7]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer

# Carregar o dataframe
df = pd.read_csv("/content/Desempenho dos estudantes em Portugal - matemática.csv", sep=",", encoding="utf-8")

# Remover a coluna '#N/D' se ela existir
df.drop(columns=["#N/D"], errors='ignore', inplace=True)

# Substituir valores '#N/D' por NaN
df.replace('#N/D', np.nan, inplace=True)

# Imputação de valores ausentes
imputer = SimpleImputer(strategy='mean')
numeric_cols = df.select_dtypes(include=[np.number]).columns
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

for col in df.select_dtypes(include=['object']).columns:
    df[col] = df[col].fillna(df[col].mode()[0])

# Transformar variáveis categóricas em numéricas
label_encoder = LabelEncoder()
for column in df.select_dtypes(include=['object']).columns:
    df[column] = label_encoder.fit_transform(df[column].astype(str))

# Regressão - Prevendo a nota final
X_r = df.drop(columns=['nota_final'])
y_r = df['nota_final']

scaler = StandardScaler()
X_r_scaled = scaler.fit_transform(X_r)

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(X_r_scaled, y_r, test_size=0.3, random_state=42)

modelo_regressao = LinearRegression()
modelo_regressao.fit(X_train_r, y_train_r)
y_pred_reg = modelo_regressao.predict(X_test_r)

modelo_arvore_r = DecisionTreeRegressor(random_state=42)
modelo_arvore_r.fit(X_train_r, y_train_r)
y_pred_arvore_r = modelo_arvore_r.predict(X_test_r)

# Classificação - Aprovado/Reprovado
y_c = (df['nota_final'] >= 10).astype(int)
X_train_c, X_test_c, y_train_c, y_test_c = train_test_split(X_r_scaled, y_c, test_size=0.3, random_state=42)

modelo_logistico = LogisticRegression()
modelo_logistico.fit(X_train_c, y_train_c)
y_pred_logistico = modelo_logistico.predict(X_test_c)

modelo_arvore_c = DecisionTreeClassifier(random_state=42)
modelo_arvore_c.fit(X_train_c, y_train_c)
y_pred_arvore_c = modelo_arvore_c.predict(X_test_c)

def avaliar_classificacao(nome, y_true, y_pred):
    print(f"\n🔹 Modelo: {nome}")
    print(f"Acurácia: {accuracy_score(y_true, y_pred):.2f}")
    print(f"Precisão: {precision_score(y_true, y_pred):.2f}")
    print(f"Recall: {recall_score(y_true, y_pred):.2f}")
    print(f"Matriz de Confusão:\n{confusion_matrix(y_true, y_pred)}")

avaliar_classificacao("Regressão Logística", y_test_c, y_pred_logistico)
avaliar_classificacao("Árvore de Decisão (Classificação)", y_test_c, y_pred_arvore_c)

r2_reg = r2_score(y_test_r, y_pred_reg)
r2_arvore_r = r2_score(y_test_r, y_pred_arvore_r)

print(f"\n🔹 Modelo de Regressão Linear - R²: {r2_reg:.2f}")
print(f"🔹 Modelo de Árvore de Decisão Regressora - R²: {r2_arvore_r:.2f}")

if r2_arvore_r > r2_reg:
    print("✅ A Árvore de Decisão Regressora teve melhor desempenho para prever a nota final.")
else:
    print("✅ A Regressão Linear teve melhor desempenho para prever a nota final.")


🔹 Modelo: Regressão Logística
Acurácia: 0.92
Precisão: 0.93
Recall: 0.95
Matriz de Confusão:
[[41  5]
 [ 4 69]]

🔹 Modelo: Árvore de Decisão (Classificação)
Acurácia: 0.84
Precisão: 0.86
Recall: 0.88
Matriz de Confusão:
[[36 10]
 [ 9 64]]

🔹 Modelo de Regressão Linear - R²: 0.79
🔹 Modelo de Árvore de Decisão Regressora - R²: 0.82
✅ A Árvore de Decisão Regressora teve melhor desempenho para prever a nota final.
